<a href="https://colab.research.google.com/github/superkisa/MaGaML/blob/main/MathRefresher/sem_Words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Word vectors**


In the previous exercise we observed that colors that we think of as similar are 'closer' to each other in RGB vector space. Is it possible to create a vector space for all English words that has this same 'closer in space is closer in meaning' property?

The answer is yes! Luckily, you don't need to create those vectors from scratch. Many researchers have made downloadable databases of pre-trained vectors. One such project is [Stanford's Global Vectors for Word Representation (GloVe)](https://nlp.stanford.edu/projects/glove/). 

These $300$-dimensional vectors are included with $\texttt{spaCy}$, and they're the vectors we'll be using in this exercise.

![cosine similarity: picture](https://d33wubrfki0l68.cloudfront.net/d2742976a92aa4d6c39f19c747ec5f56ed1cec30/3803f/images/guide-to-word-vectors-with-gensim-and-keras_files/word2vec-king-queen-vectors.png)

In [1]:
# The following will download the language model.
# Resart the runtime (Runtime -> Restart runtime) after running this cell
# (and don't run it for the second time).
!python -m spacy download en_core_web_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████▉   | 530.2 MB 4.1 kB/s eta 3:54:01
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 319, in run
    reqs, check_supported_wheels=not options.target_dir
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 104, in resolve
    req, requested_extras=()
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/resolution/resolvelib/factory.py", line 434, in make_requirement_from_install_req
    version=None,
  File "/usr/local/lib/python3.7/di

Let's load the model now:

In [1]:
import spacy

nlp = spacy.load('en_core_web_lg')

## **Word vectors: the first glance**

You can see the vector of any word in $\texttt{spaCy}$' s vocabulary using the $\texttt{vector}$ attribute:

In [2]:
# A 300-dimensional vector
len(nlp('dog').vector)

300

In [3]:
nlp('dog').vector

array([ 1.2330e+00,  4.2963e+00, -7.9738e+00, -1.0121e+01,  1.8207e+00,
        1.4098e+00, -4.5180e+00, -5.2261e+00, -2.9157e-01,  9.5234e-01,
        6.9880e+00,  5.0637e+00, -5.5726e-03,  3.3395e+00,  6.4596e+00,
       -6.3742e+00,  3.9045e-02, -3.9855e+00,  1.2085e+00, -1.3186e+00,
       -4.8886e+00,  3.7066e+00, -2.8281e+00, -3.5447e+00,  7.6888e-01,
        1.5016e+00, -4.3632e+00,  8.6480e+00, -5.9286e+00, -1.3055e+00,
        8.3870e-01,  9.0137e-01, -1.7843e+00, -1.0148e+00,  2.7300e+00,
       -6.9039e+00,  8.0413e-01,  7.4880e+00,  6.1078e+00, -4.2130e+00,
       -1.5384e-01, -5.4995e+00,  1.0896e+01,  3.9278e+00, -1.3601e-01,
        7.7732e-02,  3.2218e+00, -5.8777e+00,  6.1359e-01, -2.4287e+00,
        6.2820e+00,  1.3461e+01,  4.3236e+00,  2.4266e+00, -2.6512e+00,
        1.1577e+00,  5.0848e+00, -1.7058e+00,  3.3824e+00,  3.2850e+00,
        1.0969e+00, -8.3711e+00, -1.5554e+00,  2.0296e+00, -2.6796e+00,
       -6.9195e+00, -2.3386e+00, -1.9916e+00, -3.0450e+00,  2.48

## **Cosine similarity**

**Cosine similarity** is a common way of assessing similarity between words in NLP. It is essentially defined as the cosine of the angle between the vectors representing the words of interest.

Recall that the angle $\phi$ between two non-zero vectors $u$ and $v$ can be computed as follows:

$cos(\phi) = \frac{(u,v)}{||u||\cdot||v||}$

![](https://miro.medium.com/max/1394/1*_Bf9goaALQrS_0XkBozEiQ.png)



Define a function computing cosine similarity between two vectors.

In [4]:
import numpy as np
from math import sqrt

def cosine(v1, v2):
  # Your code here
  return np.sum(v1*v2)/(sqrt(np.sum(v1*v1))*sqrt(np.sum(v2*v2)))

Test your function by computing similarities of some random pairs of words, e.g. $dog$ and $puppy$ vs. $dog$ and $kitten$. 

In [5]:
# Your code here
print(cosine(nlp('dog').vector, nlp('puppy').vector))
print(cosine(nlp('dog').vector, nlp('kitten').vector))

0.8107667523600081
0.6515031235181183


## **Loading the text**

Let's load the full text of *Alice in Wonderland*. It will serve us as a corpus of English words.

In [6]:
import requests

# Alice in Wonderland
response = requests.get('https://www.gutenberg.org/files/11/11-0.txt')

# If you prefer Dracula, load this instead:
#response = requests.get('https://www.gutenberg.org/cache/epub/345/pg345.txt')

# Extracting separate words from the text
doc = nlp(response.text)
tokens = list(set([w.text for w in doc if w.is_alpha]))

Check out the content of $\texttt{tokens}$ now.

In [7]:
tokens

['TRADEMARK',
 'verse',
 'favoured',
 'Section',
 'twenty',
 'seaside',
 'pleasanter',
 'neck',
 'It',
 'same',
 'pigeon',
 'grunt',
 'Take',
 'clamour',
 'hollow',
 'afore',
 'patted',
 'breathe',
 'Ground',
 'foolish',
 'better',
 'We',
 'curls',
 'Mercia',
 'paint',
 'brave',
 'clear',
 'lifted',
 'Paris',
 'Why',
 'else',
 'States',
 'Her',
 'maintaining',
 'King',
 'zigzag',
 'licking',
 'chimney',
 'files',
 'pretending',
 'round',
 'goose',
 'kissed',
 'distribute',
 'trees',
 'Heads',
 'daughter',
 'denying',
 'FITNESS',
 'miles',
 'dry',
 'remembered',
 'think',
 'bent',
 'choke',
 'plate',
 'quietly',
 'centre',
 'feather',
 'comfortable',
 'DISTRIBUTE',
 'proprietary',
 'pebbles',
 'figure',
 'lobster',
 'traps',
 'secret',
 'commercial',
 'grow',
 'financial',
 'copied',
 'listen',
 'do',
 'succeeded',
 'user',
 'LIABLE',
 'INCLUDING',
 'country',
 'tight',
 'shrill',
 'attending',
 'Before',
 'Rome',
 'idea',
 'becoming',
 'sneezed',
 'these',
 'paper',
 'ago',
 'belt',
 '

Define a function that takes a word and lists the $n$ most similar words in our corpus.

In [22]:
import pandas as pd
def spacy_closest(tokens, new_vec, n=10):
  # Your code here
    d = {'tokens': tokens,}
    df = pd.DataFrame(data=d)
    df['tokens'] = tokens
    df['cos_token'] = [cosine(new_vec, nlp(token_i).vector) for token_i in tokens]
    df['abs_cos_token'] = [abs(cos_i) for cos_i in df['cos_token']]
    return df.sort_values(by="abs_cos_token",)[:n]

Try to find words similar to some random words, e.g. $good$.

In [23]:
spacy_closest(tokens, nlp('good').vector)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  


,tokens,cos_token,abs_cos_token
1038,DIRECT,-0.000424,0.000424
1239,grin,-0.000507,0.000507
376,USE,0.000618,0.000618
2173,woke,0.000618,0.000618
954,WARRANTY,-0.000843,0.000843
1535,Has,0.000870,0.000870
1911,Zealand,-0.001329,0.001329
980,locations,-0.001458,0.001458
814,threw,0.001668,0.001668
2438,paw,0.002174,0.002174


You can also get creative and search for combinations of words. For example, what is similar to $king - man + woman$? 

In [24]:
# Your code here
spacy_closest(tokens, nlp('king').vector - nlp('man').vector + nlp('woman').vector)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  


,tokens,cos_token,abs_cos_token
2089,May,0.000215,0.000215
2477,Author,0.000225,0.000225
2618,suet,-0.000310,0.000310
1995,up,0.000323,0.000323
1882,nice,0.000408,0.000408
2046,BREACH,-0.000448,0.000448
240,Ann,0.000455,0.000455
233,toffee,0.000542,0.000542
401,lose,-0.000731,0.000731
683,nervous,-0.000771,0.000771


## **Sentence vectors**

We can also construct a vector representation for the whole sentence. For example, we can define it as an *average* of the   vectors representing the words in it.

Let's take a random sentence *My favorite food is strawberry ice cream* and construct its vector representation.

In [37]:
sent = nlp('My favorite food is strawberry ice cream.')
np.sum(sent[:-1].vector)/(len(sent) - 1)

# Your code here
# sentv ...

-6.100927625383649

Let's also extract sentences (as opposed to individual words) from our corpus:

In [38]:
sents = list(doc.sents)

Define a function that takes a random sentence and lists $n$ most similar sentences from our corpus.

In [ ]:
def spacy_closest_sent(sentences, input_vec, n=10):
  # Your code here
  d = {'sentences': sentences,}
  df = pd.DataFrame(data=d)
  df['sentences'] = sentences
  df['cos_token'] = [cosine(input_vec, nlp(sentence_i).vector) for sentence_i in sentences]
  df['abs_cos_token'] = [abs(cos_i) for cos_i in df['cos_token']]
  return df.sort_values(by="abs_cos_token",)[:n]

Let's try it out!

In [ ]:
for s in spacy_closest_sent(sents, sentv, n=10):
  print(s)
  print('\n---')

## **References**

This notebook is inspired by a [tutorial by Allison Parrish](https://gist.github.com/aparrish/2f562e3737544cf29aaf1af30362f469).